In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('MCA_Thesis (2).pdf')
docs = loader.load()
docs

[Document(page_content='An Empirical Study with Pre Trained and Attention Model\non Food Disease Classification\nA Project submitted in partial fulfillment of requirements\nFor the award of the degree of\nMaster of Computer Application\nby\nSohom Roy Choudhury\nRegn. No.- 221040510049 Exam Roll No. - 12022010010038\nRishav Bhattacharjee\nRegn. No.- 221040510036 Exam Roll No. - 12022010010008\nAyan Batabyal\nRegn. No.- 221040510011 Exam Roll No. - 12022010010003\nunder the supervision of\nProf. Supratim Ghosh\nAssistant Professor\n&\nProf. Dr. Priti Deb\nAssistant Professor\nDepartment of Computer Application\nInstitute of Engineering & Management\nKolkata, West Bengal, India\n2022-2024', metadata={'source': 'MCA_Thesis (2).pdf', 'page': 0}),
 Document(page_content='Declaration Certificate\nThis is to certify that the work presented in the thesis entitled “An Empirical Study with\nPre-Trained and Attention Model on Food Disease Classification” in partial fulfilment of\nthe requirement f

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents[:5]

[Document(page_content='An Empirical Study with Pre Trained and Attention Model\non Food Disease Classification\nA Project submitted in partial fulfillment of requirements\nFor the award of the degree of\nMaster of Computer Application\nby\nSohom Roy Choudhury\nRegn. No.- 221040510049 Exam Roll No. - 12022010010038\nRishav Bhattacharjee\nRegn. No.- 221040510036 Exam Roll No. - 12022010010008\nAyan Batabyal\nRegn. No.- 221040510011 Exam Roll No. - 12022010010003\nunder the supervision of\nProf. Supratim Ghosh\nAssistant Professor\n&\nProf. Dr. Priti Deb\nAssistant Professor\nDepartment of Computer Application\nInstitute of Engineering & Management\nKolkata, West Bengal, India\n2022-2024', metadata={'source': 'MCA_Thesis (2).pdf', 'page': 0}),
 Document(page_content='Declaration Certificate\nThis is to certify that the work presented in the thesis entitled “An Empirical Study with\nPre-Trained and Attention Model on Food Disease Classification” in partial fulfilment of\nthe requirement f

In [4]:
len(documents)

116

In [5]:
from langchain_community.embeddings import OllamaEmbeddings # Or OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

db_faiss = FAISS.from_documents(documents,OllamaEmbeddings())

In [6]:
query="What is the summary of this paper"
res = db_faiss.similarity_search(query)
res

[Document(page_content='MobileNet 83 45 32\nResNet101V2 82 35 32\nDenseNet121 79 45 32\nAs seen in the below Confusion matrix in Figure 6.7, MobileNet was able to identify Strawberry’s\nLeaf Spot class correctly the best with 95% of the times the result being True Positive. MobileNet\nperformed the worst against Strawberry’s Gray Mold class with 52% True Positivity. Beans’ Angular\nLeaf spot class tricked MobileNet the most with 30% of the time it being classified as Beans’ Rust\nclass. As seen in the below Confusion matrix in Figure 6.8, ResNet101V2 was able to identify\nStrawberry’s Leaf Spot class correctly the best with 99% of the times the result being True Positive.\nResNet101V2 performed the worst against Strawberry’s Gray Mold class with 50% True Positivity.\nStrawberry’s Gray Mold class tricked ResNet101V2 the most with 32% of the time it being classified\nas Strawberry’s Powdery Mildew Fruit class. As seen in the below Confusion matrix in Figure 6.9,', metadata={'source': 'MC

In [7]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")
llm

Ollama()

### Design ChatPrompt Template

In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
    Answer the questions based only on the provided context.
                                          Think step by step before providing a detailed answer.
                                          I will tip you $1000 if the answer finds the answer useful.
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}
""")

## Chain Introduction
#### Create Stuff Document Chain

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [10]:
retriever = db_faiss.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000221A22C4690>)

#### Retrieval chain

In [11]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [12]:
response = retrieval_chain.invoke({"input":"The primary objective is to assess their efficacy in accurately classifying a broad"})

{'input': 'The primary objective is to assess their efficacy in accurately classifying a broad',
 'context': [Document(page_content='result, a small sample of images ended up being misclassified due to the presence of residual noise.\nThe presence of noise in images can significantly impact the performance of machine learning\nmodels, particularly in tasks like image classification, where accurate feature extraction is crucial.\nNoise can introduce spurious patterns and distortions, leading to erroneous predictions by the\nmodel. To address this issue in future efforts, it may be necessary to refine the noise removal\nprocess by experimenting with different thresholding techniques or adaptive thresholding methods.\nAdditionally, incorporating data augmentation techniques during preprocessing, such as random\nrotations, translations, or adding simulated noise, could help improve the robustness of the model\nto noise artifacts. Furthermore, conducting thorough analysis and validation of 

In [ ]:
response['answer']